In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [7]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    ####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.ones_like(gQs2))) # 0-1
    # ##################################################### repeatable!
    # next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
    #                                reuse=True)
    # next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
    #                             reuse=True)
    # gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
    #                      action_size=action_size, reuse=True)
    # dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    # gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                 labels=tf.zeros_like(gQs3))) # 0-1
    # aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                  labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [8]:
def model_opt(a_loss, d_loss, a_loss2, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    #g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        d_loss=self.d_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [10]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [11]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [13]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
# gamma = 0.99                   # future reward discount

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              d_learning_rate=d_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [15]:
state = env.reset()
total_reward = 0
num_step = 0
rate = -1
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset
        rate = -1 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode
        rate = -1 # unrated

        # Sampling
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            if done is True:
                break
        
        # Rating
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
        # Training
        batch = memory.buffer
        for idx in range(memory_size// batch_size):
        # while True:
            #idx = np.random.choice(np.arange(memory_size// batch_size))
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            states = states[rates == np.max(rates)]
            actions = actions[rates == np.max(rates)]
            next_states = next_states[rates == np.max(rates)]
            #rewards = rewards[rates == np.max(rates)]
            dones = dones[rates == np.max(rates)]
            rates = rates[rates == np.max(rates)]
            #if np.count_nonzero(dones)==1 and len(dones) > 1 and np.max(rates) > 0:
            #if len(dones) > 1:
            #if np.count_nonzero(dones)==0 and len(dones) > 1:
            if np.count_nonzero(dones)==0 and len(dones) > 1 and np.max(rates) > 0:
                aloss, _ = sess.run([model.a_loss, model.a_opt],
                                    feed_dict = {model.states: states, 
                                                model.actions: actions,
                                                model.next_states: next_states,
                                                #model.rewards: rewards,
                                                #model.dones: dones,
                                                model.rates: rates})
                dloss, _ = sess.run([model.d_loss, model.d_opt],
                                      feed_dict = {model.states: states, 
                                                   model.actions: actions,
                                                   model.next_states: next_states,
                                                   #model.rewards: rewards,
                                                   #model.dones: dones,
                                                   model.rates: rates})
                aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                     feed_dict = {model.states: states, 
                                                  model.actions: actions,
                                                  model.next_states: next_states,
                                                  #model.rewards: rewards,
                                                  #model.dones: dones,
                                                  model.rates: rates})
                aloss_batch.append(aloss)
                dloss_batch.append(dloss)
                aloss2_batch.append(aloss2)

        # Printing
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:14.0000 R:14.0000 rate:0.0280 aloss:1.3906 dloss:2.2502 aloss2:0.6326 exploreP:0.9986
Episode:1 meanR:23.5000 R:33.0000 rate:0.0660 aloss:1.3915 dloss:2.1372 aloss2:0.6745 exploreP:0.9954
Episode:2 meanR:19.6667 R:12.0000 rate:0.0240 aloss:1.3902 dloss:2.2127 aloss2:0.6452 exploreP:0.9942
Episode:3 meanR:20.7500 R:24.0000 rate:0.0480 aloss:1.3939 dloss:2.5148 aloss2:0.5556 exploreP:0.9918
Episode:4 meanR:20.4000 R:19.0000 rate:0.0380 aloss:1.3963 dloss:2.4957 aloss2:0.5527 exploreP:0.9900
Episode:5 meanR:20.3333 R:20.0000 rate:0.0400 aloss:1.3884 dloss:2.1587 aloss2:0.6645 exploreP:0.9880
Episode:6 meanR:20.0000 R:18.0000 rate:0.0360 aloss:1.3905 dloss:2.1869 aloss2:0.6545 exploreP:0.9862
Episode:7 meanR:19.3750 R:15.0000 rate:0.0300 aloss:1.3889 dloss:2.1742 aloss2:0.6572 exploreP:0.9848
Episode:8 meanR:19.0000 R:16.0000 rate:0.0320 aloss:1.3876 dloss:2.1965 aloss2:0.6513 exploreP:0.9832
Episode:9 meanR:19.0000 R:19.0000 rate:0.0380 aloss:1.3917 dloss:2.2032 aloss2:0.6

Episode:80 meanR:24.1852 R:44.0000 rate:0.0880 aloss:1.3841 dloss:1.8863 aloss2:0.7660 exploreP:0.8239
Episode:81 meanR:24.1829 R:24.0000 rate:0.0480 aloss:1.3828 dloss:1.8873 aloss2:0.7669 exploreP:0.8219
Episode:82 meanR:24.1687 R:23.0000 rate:0.0460 aloss:1.3858 dloss:1.8966 aloss2:0.7580 exploreP:0.8201
Episode:83 meanR:24.0595 R:15.0000 rate:0.0300 aloss:1.3813 dloss:1.9351 aloss2:0.7435 exploreP:0.8188
Episode:84 meanR:24.0118 R:20.0000 rate:0.0400 aloss:1.3790 dloss:1.8898 aloss2:0.7649 exploreP:0.8172
Episode:85 meanR:24.0116 R:24.0000 rate:0.0480 aloss:1.3773 dloss:1.9334 aloss2:0.7439 exploreP:0.8153
Episode:86 meanR:23.9080 R:15.0000 rate:0.0300 aloss:1.3888 dloss:2.0614 aloss2:0.6930 exploreP:0.8141
Episode:87 meanR:23.8977 R:23.0000 rate:0.0460 aloss:1.3947 dloss:2.0632 aloss2:0.6902 exploreP:0.8122
Episode:88 meanR:23.7978 R:15.0000 rate:0.0300 aloss:1.3826 dloss:1.8658 aloss2:0.7755 exploreP:0.8110
Episode:89 meanR:24.1000 R:51.0000 rate:0.1020 aloss:1.3817 dloss:1.8890 

Episode:159 meanR:33.9500 R:20.0000 rate:0.0400 aloss:1.3780 dloss:1.6357 aloss2:0.9083 exploreP:0.6247
Episode:160 meanR:33.9700 R:24.0000 rate:0.0480 aloss:1.3788 dloss:1.6513 aloss2:0.8934 exploreP:0.6232
Episode:161 meanR:33.8700 R:18.0000 rate:0.0360 aloss:1.3774 dloss:1.6086 aloss2:0.9186 exploreP:0.6221
Episode:162 meanR:33.8700 R:20.0000 rate:0.0400 aloss:1.3784 dloss:1.6108 aloss2:0.9149 exploreP:0.6209
Episode:163 meanR:34.1800 R:43.0000 rate:0.0860 aloss:1.3768 dloss:1.5724 aloss2:0.9371 exploreP:0.6183
Episode:164 meanR:34.2600 R:29.0000 rate:0.0580 aloss:1.3761 dloss:1.6068 aloss2:0.9175 exploreP:0.6165
Episode:165 meanR:34.5000 R:39.0000 rate:0.0780 aloss:1.3808 dloss:1.5823 aloss2:0.9290 exploreP:0.6141
Episode:166 meanR:34.5100 R:31.0000 rate:0.0620 aloss:1.3783 dloss:1.5977 aloss2:0.9211 exploreP:0.6123
Episode:167 meanR:35.0100 R:67.0000 rate:0.1340 aloss:1.3724 dloss:1.5898 aloss2:0.9265 exploreP:0.6082
Episode:168 meanR:35.3100 R:72.0000 rate:0.1440 aloss:1.3768 dlo

Episode:238 meanR:29.2700 R:26.0000 rate:0.0520 aloss:1.3702 dloss:1.4697 aloss2:0.9744 exploreP:0.5073
Episode:239 meanR:29.1900 R:28.0000 rate:0.0560 aloss:1.3659 dloss:1.4863 aloss2:0.9649 exploreP:0.5059
Episode:240 meanR:29.1800 R:14.0000 rate:0.0280 aloss:1.3660 dloss:1.4944 aloss2:0.9519 exploreP:0.5052
Episode:241 meanR:29.1500 R:16.0000 rate:0.0320 aloss:1.3994 dloss:1.5029 aloss2:0.9435 exploreP:0.5044
Episode:242 meanR:29.0500 R:15.0000 rate:0.0300 aloss:1.3724 dloss:1.5016 aloss2:0.9503 exploreP:0.5037
Episode:243 meanR:28.4500 R:18.0000 rate:0.0360 aloss:1.3712 dloss:1.4992 aloss2:0.9523 exploreP:0.5028
Episode:244 meanR:28.3500 R:13.0000 rate:0.0260 aloss:1.3922 dloss:1.5203 aloss2:0.9427 exploreP:0.5022
Episode:245 meanR:28.3100 R:25.0000 rate:0.0500 aloss:1.3722 dloss:1.5019 aloss2:0.9534 exploreP:0.5009
Episode:246 meanR:28.2400 R:20.0000 rate:0.0400 aloss:1.3589 dloss:1.4971 aloss2:0.9562 exploreP:0.5000
Episode:247 meanR:27.5700 R:24.0000 rate:0.0480 aloss:1.3620 dlo

Episode:317 meanR:26.2500 R:26.0000 rate:0.0520 aloss:1.3510 dloss:1.3289 aloss2:1.0724 exploreP:0.4116
Episode:318 meanR:26.3400 R:31.0000 rate:0.0620 aloss:1.3629 dloss:1.3007 aloss2:1.0883 exploreP:0.4104
Episode:319 meanR:26.2800 R:18.0000 rate:0.0360 aloss:1.3732 dloss:1.2977 aloss2:1.0941 exploreP:0.4097
Episode:320 meanR:26.2800 R:16.0000 rate:0.0320 aloss:1.3720 dloss:1.3019 aloss2:1.0927 exploreP:0.4090
Episode:321 meanR:26.2300 R:16.0000 rate:0.0320 aloss:1.3667 dloss:1.2960 aloss2:1.1066 exploreP:0.4084
Episode:322 meanR:26.1300 R:18.0000 rate:0.0360 aloss:1.3707 dloss:1.2996 aloss2:1.0913 exploreP:0.4077
Episode:323 meanR:26.2600 R:31.0000 rate:0.0620 aloss:1.3913 dloss:1.3085 aloss2:1.0773 exploreP:0.4064
Episode:324 meanR:25.8900 R:10.0000 rate:0.0200 aloss:1.3671 dloss:1.3007 aloss2:1.0917 exploreP:0.4060
Episode:325 meanR:25.9300 R:25.0000 rate:0.0500 aloss:1.3773 dloss:1.3061 aloss2:1.0860 exploreP:0.4050
Episode:326 meanR:26.0200 R:33.0000 rate:0.0660 aloss:1.3687 dlo

Episode:396 meanR:20.2400 R:21.0000 rate:0.0420 aloss:1.4004 dloss:1.1258 aloss2:1.2297 exploreP:0.3542
Episode:397 meanR:20.1500 R:18.0000 rate:0.0360 aloss:1.4694 dloss:1.1580 aloss2:1.2273 exploreP:0.3536
Episode:398 meanR:20.1700 R:19.0000 rate:0.0380 aloss:1.3903 dloss:1.1687 aloss2:1.2508 exploreP:0.3530
Episode:399 meanR:20.1700 R:23.0000 rate:0.0460 aloss:1.3870 dloss:1.1449 aloss2:1.2292 exploreP:0.3522
Episode:400 meanR:20.1600 R:18.0000 rate:0.0360 aloss:1.3848 dloss:1.1454 aloss2:1.2329 exploreP:0.3516
Episode:401 meanR:20.0200 R:14.0000 rate:0.0280 aloss:1.3752 dloss:1.1408 aloss2:1.2334 exploreP:0.3511
Episode:402 meanR:20.0100 R:20.0000 rate:0.0400 aloss:1.3791 dloss:1.1612 aloss2:1.2428 exploreP:0.3504
Episode:403 meanR:19.9900 R:15.0000 rate:0.0300 aloss:1.4325 dloss:1.1509 aloss2:1.2303 exploreP:0.3499
Episode:404 meanR:19.8100 R:19.0000 rate:0.0380 aloss:1.3803 dloss:1.1558 aloss2:1.2443 exploreP:0.3492
Episode:405 meanR:19.8400 R:18.0000 rate:0.0360 aloss:1.4286 dlo

Episode:475 meanR:18.9600 R:22.0000 rate:0.0440 aloss:1.3843 dloss:0.9634 aloss2:1.4511 exploreP:0.3074
Episode:476 meanR:19.1500 R:43.0000 rate:0.0860 aloss:1.3862 dloss:0.9409 aloss2:1.4929 exploreP:0.3061
Episode:477 meanR:19.1800 R:20.0000 rate:0.0400 aloss:1.3851 dloss:0.9485 aloss2:1.4820 exploreP:0.3055
Episode:478 meanR:19.0800 R:22.0000 rate:0.0440 aloss:1.4162 dloss:0.9343 aloss2:1.4666 exploreP:0.3049
Episode:479 meanR:18.8600 R:19.0000 rate:0.0380 aloss:1.4259 dloss:0.9650 aloss2:1.4523 exploreP:0.3043
Episode:480 meanR:18.9100 R:19.0000 rate:0.0380 aloss:1.3859 dloss:0.9398 aloss2:1.4726 exploreP:0.3037
Episode:481 meanR:18.9400 R:16.0000 rate:0.0320 aloss:1.3804 dloss:0.9375 aloss2:1.4817 exploreP:0.3033
Episode:482 meanR:18.9700 R:22.0000 rate:0.0440 aloss:1.3866 dloss:0.9481 aloss2:1.4765 exploreP:0.3026
Episode:483 meanR:19.0600 R:24.0000 rate:0.0480 aloss:1.4056 dloss:0.9200 aloss2:1.4827 exploreP:0.3019
Episode:484 meanR:19.0000 R:23.0000 rate:0.0460 aloss:1.3720 dlo

Episode:554 meanR:20.3800 R:19.0000 rate:0.0380 aloss:1.3942 dloss:0.8704 aloss2:1.6763 exploreP:0.2624
Episode:555 meanR:20.3900 R:18.0000 rate:0.0360 aloss:1.3963 dloss:0.8536 aloss2:1.6821 exploreP:0.2619
Episode:556 meanR:20.3300 R:21.0000 rate:0.0420 aloss:1.3802 dloss:0.8455 aloss2:1.6863 exploreP:0.2614
Episode:557 meanR:20.3900 R:22.0000 rate:0.0440 aloss:1.4325 dloss:0.8548 aloss2:1.6693 exploreP:0.2609
Episode:558 meanR:20.4100 R:20.0000 rate:0.0400 aloss:1.3865 dloss:0.8382 aloss2:1.7106 exploreP:0.2604
Episode:559 meanR:20.3900 R:21.0000 rate:0.0420 aloss:1.3730 dloss:0.8543 aloss2:1.7127 exploreP:0.2598
Episode:560 meanR:20.5300 R:25.0000 rate:0.0500 aloss:1.3729 dloss:0.8063 aloss2:1.7146 exploreP:0.2592
Episode:561 meanR:20.6000 R:20.0000 rate:0.0400 aloss:1.3740 dloss:0.8051 aloss2:1.7173 exploreP:0.2587
Episode:562 meanR:20.6400 R:22.0000 rate:0.0440 aloss:1.3871 dloss:0.8075 aloss2:1.7277 exploreP:0.2582
Episode:563 meanR:20.6100 R:22.0000 rate:0.0440 aloss:1.3573 dlo

Episode:633 meanR:25.9800 R:63.0000 rate:0.1260 aloss:1.3733 dloss:0.7088 aloss2:1.9476 exploreP:0.2137
Episode:634 meanR:26.0200 R:24.0000 rate:0.0480 aloss:1.3784 dloss:0.7143 aloss2:1.9602 exploreP:0.2132
Episode:635 meanR:26.0600 R:22.0000 rate:0.0440 aloss:1.3495 dloss:0.7247 aloss2:1.9762 exploreP:0.2127
Episode:636 meanR:26.0700 R:24.0000 rate:0.0480 aloss:1.3497 dloss:0.6745 aloss2:1.9962 exploreP:0.2123
Episode:637 meanR:26.4800 R:63.0000 rate:0.1260 aloss:1.3930 dloss:0.6767 aloss2:1.9679 exploreP:0.2110
Episode:638 meanR:26.6600 R:40.0000 rate:0.0800 aloss:1.3902 dloss:0.6868 aloss2:1.9643 exploreP:0.2102
Episode:639 meanR:26.7700 R:28.0000 rate:0.0560 aloss:1.3800 dloss:0.6999 aloss2:1.9692 exploreP:0.2096
Episode:640 meanR:26.7500 R:19.0000 rate:0.0380 aloss:1.3669 dloss:0.6827 aloss2:1.9735 exploreP:0.2092
Episode:641 meanR:27.1800 R:64.0000 rate:0.1280 aloss:1.3506 dloss:0.7095 aloss2:1.9750 exploreP:0.2080
Episode:642 meanR:27.1300 R:19.0000 rate:0.0380 aloss:1.3218 dlo

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
# eps, arr = np.array(dloss_list).T
# smoothed_arr = running_mean(arr, 10)
# plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
# plt.plot(eps, arr, color='grey', alpha=0.3)
# plt.xlabel('Episode')
# plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.